In [1]:
import os
import zipfile

import numpy as np
import pandas as pd

from pandas import DataFrame
from scipy.stats.mstats import winsorize

from Constant import Constants as const
from OrganizeData.step02_merge_all_financial_data import sort_csmar_data

# Sort Finance Cost Data

In [25]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '1990_2023_Income Statement.zip'), 'r') as zip_ref:
    with zip_ref.open('FS_Comins.csv') as csv_file:
        finidx_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip',
                                           usecols=['Stkcd', 'Accper', 'B001211000', 'B001211101',
                                                    'Bbd1102203']).rename(
            columns={'B001211000': 'FinaExpense', 'B001211101': 'InterestExpense',
                     'Bbd1102203': 'InterestExpenses'}).dropna(
            subset=['FinaExpense', 'InterestExpense', 'InterestExpenses'], how='all')
        finidx_df: DataFrame = sort_csmar_data(finidx_df)

In [27]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '1990_2023_Balance Sheet.zip'), 'r') as zip_ref:
    with zip_ref.open('FS_Combas.csv') as csv_file:
        fscombas_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip',
                                             usecols=['Stkcd', 'Accper', 'A001000000', 'A002000000']).rename(
            columns={'A001000000': 'at', 'A002000000': 'lt'})
        fscombas_df: DataFrame = sort_csmar_data(fscombas_df)

In [28]:
fc_df: DataFrame = finidx_df.merge(fscombas_df, on=[const.TICKER, const.YEAR], how='left')
fc_df.sort_values(by=[const.TICKER, const.YEAR], ascending=True, inplace=True)
fc_df.loc[:, 'lag_at'] = fc_df.groupby(const.TICKER)['at'].shift(1)
fc_df.loc[:, 'lag_lt'] = fc_df.groupby(const.TICKER)['lt'].shift(1)

fc_df['fe_at'] = fc_df['FinaExpense'] / fc_df['at']
fc_df['fe_lt'] = fc_df['FinaExpense'] / fc_df['lt']
fc_df['fe_lat'] = fc_df['FinaExpense'] / fc_df['lag_at']
fc_df['fe_llt'] = fc_df['FinaExpense'] / fc_df['lag_lt']

fc_df['ie_at'] = fc_df['InterestExpense'] / fc_df['at']
fc_df['ie_lt'] = fc_df['InterestExpense'] / fc_df['lt']
fc_df['ie_lat'] = fc_df['InterestExpense'] / fc_df['lag_at']
fc_df['ie_llt'] = fc_df['InterestExpense'] / fc_df['lag_lt']

fc_df['ies_at'] = fc_df['InterestExpenses'] / fc_df['at']
fc_df['ies_lt'] = fc_df['InterestExpenses'] / fc_df['lt']
fc_df['ies_lat'] = fc_df['InterestExpenses'] / fc_df['lag_at']
fc_df['ies_llt'] = fc_df['InterestExpenses'] / fc_df['lag_lt']

In [30]:
fc_df.replace([np.inf, -np.inf], np.nan, inplace=True)
fc_df.describe()

,tic,InterestExpenses,FinaExpense,InterestExpense,year,at,lt,lag_at,lag_lt,fe_at,...,fe_lat,fe_llt,ie_at,ie_lt,ie_lat,ie_llt,ies_at,ies_lt,ies_lat,ies_llt
count,70379.000000,1.502800e+04,6.963400e+04,2.515700e+04,70379.000000,7.022100e+04,7.021600e+04,6.455100e+04,6.454600e+04,69474.000000,...,63884.000000,63876.000000,25156.000000,25157.000000,23632.000000,23631.000000,15028.000000,15022.000000,14141.000000,14136.000000
mean,339479.682377,2.598846e+09,8.321386e+07,1.466576e+08,2013.669461,3.990082e+10,3.324762e+10,3.770796e+10,3.145459e+10,0.032418,...,0.013242,0.347566,0.010208,0.348100,0.011657,0.454932,0.001353,0.560346,0.002210,0.136404
std,292799.994027,2.377611e+10,5.146331e+08,6.943180e+08,7.627352,7.052894e+11,6.488194e+11,6.652362e+11,6.119789e+11,5.191898,...,0.189967,31.158398,0.024119,16.262805,0.024459,20.735176,0.010305,51.070826,0.057734,10.428371
min,1.000000,-3.280945e+04,-8.605000e+09,-5.812000e+09,1991.000000,0.000000e+00,-2.033024e+06,0.000000e+00,-2.033024e+06,-0.181792,...,-0.383092,-1526.823132,-0.058643,-1.027385,-0.047658,-1.033818,-0.000062,-0.000221,-0.000058,-0.000174
25%,2230.000000,0.000000e+00,-8.451223e+05,2.697814e+06,2009.000000,1.009060e+09,2.429361e+08,9.678193e+08,2.330868e+08,-0.000650,...,-0.000595,-0.002568,0.001282,0.005457,0.001634,0.007273,0.000000,0.000000,0.000000,0.000000
50%,300651.000000,0.000000e+00,8.716583e+06,1.631878e+07,2016.000000,2.155166e+09,6.688244e+08,2.055436e+09,6.331645e+08,0.005445,...,0.006656,0.018614,0.005213,0.015677,0.006483,0.020578,0.000000,0.000000,0.000000,0.000000
75%,600713.000000,0.000000e+00,4.065497e+07,6.965684e+07,2020.000000,5.102345e+09,1.975373e+09,4.800939e+09,1.823902e+09,0.015247,...,0.017624,0.040931,0.012689,0.032190,0.014865,0.039401,0.000000,0.000000,0.000000,0.000000
max,900957.000000,5.548190e+11,2.781600e+10,3.040900e+10,2023.000000,4.243795e+13,3.895225e+13,3.773929e+13,3.439366e+13,1364.361569,...,29.131217,5345.976906,1.570772,1568.057792,1.032464,1883.610966,0.353779,6022.492599,5.261082,1097.915367


In [29]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.OUTPUT_PATH, '20241020_cc_reg_data_v2.dta'))
fc_useful_df: DataFrame = fc_df.loc[:,
                          [const.TICKER, const.YEAR, 'fe_at', 'fe_lat', 'fe_llt', 'fe_lt', 'ie_at', 'ie_lat', 'ie_lt',
                           'ie_llt', 'ies_at', 'ies_lt', 'ies_lat', 'ies_llt']].copy()
reg_df2: DataFrame = reg_df.merge(fc_useful_df, on=[const.TICKER, const.YEAR], how='left')

fc_useful_df[const.YEAR] -= 1
reg_df2: DataFrame = reg_df2.merge(fc_useful_df, on=[const.TICKER, const.YEAR], how='left', suffixes=('', '_1'))

In [31]:
for key in ['fe_at', 'fe_lat', 'fe_llt', 'fe_lt', 'ie_at', 'ie_lat', 'ie_llt', 'ie_llt', 'ies_at', 'ies_lt', 'ies_lat',
            'ies_llt']:
    reg_df2.loc[reg_df2[key].notnull(), key] = winsorize(reg_df2[key].dropna(), limits=(0.005, 0.005))
    # fc_useful_df.loc[fc_useful_df[key].notnull(), key] = winsorize(fc_useful_df[key].dropna(), limits=(0.005, 0.005))
    reg_df2.loc[reg_df2[f'{key}_1'].notnull(), f'{key}_1'] = winsorize(reg_df2[f'{key}_1'].dropna(),
                                                                       limits=(0.005, 0.005))

In [32]:
reg_df2.to_stata(
    os.path.join(const.OUTPUT_PATH, '20241026_cc_reg_data.dta'), write_index=False, version=119
)

# Construct Bond level data

In [3]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '债券基本情况表.zip'), 'r') as zip_ref:
    with zip_ref.open('BND_Bndinfo.csv') as csv_file:
        bnd_info_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip').dropna(subset=['IssSymbol'])

C:\Users\wangy\AppData\Local\Temp\ipykernel_20360\4132258909.py:3: DtypeWarning: Columns (0,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  bnd_info_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip').dropna(subset=['IssSymbol'])


In [12]:
df_gov = pd.read_excel(os.path.join(const.DATABASE_PATH, 'resset', '1981_2024_国债数据.xls'))

In [29]:
# Convert dates to datetime format
bnd_info_df['Listdt'] = pd.to_datetime(bnd_info_df['Listdt'])
df_gov['首次信息发布时间_IInfoPubDt'] = pd.to_datetime(df_gov['首次信息发布时间_IInfoPubDt'])

df_gov2: DataFrame = df_gov[df_gov['初始票面年利率(%)_CoupRt'] > 0].copy()

In [17]:
from datetime import timedelta


# Function to match the government bond to corporate bond based on similar issuance time and period
def match_bonds(corp_row, gov_bonds):
    # Filter government bonds within one year of the corporate bond issuance date
    filtered_gov_bonds = gov_bonds[
        (gov_bonds['首次信息发布时间_IInfoPubDt'] >= corp_row['Listdt'] - timedelta(days=365)) &
        (gov_bonds['首次信息发布时间_IInfoPubDt'] <= corp_row['Listdt'] + timedelta(days=365))]

    if filtered_gov_bonds.empty:
        return None

    issuance_time_diff = abs(filtered_gov_bonds['首次信息发布时间_IInfoPubDt'] - corp_row['Listdt'])
    issuance_period_diff = abs(filtered_gov_bonds['债券期限_年(年)_Maturity'] - corp_row['Term'])
    total_diff = issuance_time_diff.dt.days + issuance_period_diff * 365

    matched_gov_bond = filtered_gov_bonds.loc[total_diff.idxmin()]
    return matched_gov_bond['初始票面年利率(%)_CoupRt']

In [30]:
# Add government bond interest rate to corporate bonds
bnd_info_df['GovInterestRate'] = bnd_info_df.apply(lambda x: match_bonds(x, df_gov2), axis=1)

In [24]:
corp_row = bnd_info_df.loc[68]
# Filter government bonds within one year of the corporate bond issuance date
filtered_gov_bonds = df_gov[(df_gov['首次信息发布时间_IInfoPubDt'] >= corp_row['Listdt'] - timedelta(days=365)) &
                            (df_gov['首次信息发布时间_IInfoPubDt'] <= corp_row['Listdt'] + timedelta(days=365))]

In [26]:
issuance_time_diff = abs(filtered_gov_bonds['首次信息发布时间_IInfoPubDt'] - corp_row['Listdt'])
issuance_period_diff = abs(filtered_gov_bonds['债券期限_年(年)_Maturity'] - corp_row['Term'])
total_diff = issuance_time_diff.dt.days + issuance_period_diff * 365

matched_gov_bond = filtered_gov_bonds.loc[total_diff.idxmin()]

In [36]:
import datetime

bnd_info_df['Spread'] = bnd_info_df['Intrrate'] - bnd_info_df['GovInterestRate']
for key in ['Crdeem', 'Crtsell', 'Creplm']:
    bnd_info_df[key].replace({'Y': 1, 'N': 0}, inplace=True)

bnd_info_df.drop(['Basrted', 'Bemkrate'], axis=1, inplace=True)
bnd_info_df.loc[:, 'isPut'] = (bnd_info_df['CallOrPut'] == 'P').astype(int)
bnd_info_df.loc[:, 'isCall'] = (bnd_info_df['CallOrPut'] == 'C').astype(int)

bnd_info_df.loc[:, 'Post'] = bnd_info_df['Listdt'].apply(lambda x: int(x > datetime.datetime(2014, 5, 19)))

C:\Users\wangy\AppData\Local\Temp\ipykernel_20360\398060761.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bnd_info_df[key].replace({'Y': 1, 'N': 0}, inplace=True)


In [52]:
bnd_info_df.keys()

Index(['Liscd', 'Abbrnme', 'Sctcd', 'Listdt', 'SecurityID', 'Varsortcd',
       'Varsort', 'Orgid', 'IssSymbol', 'Acisuquty', 'Pooprc', 'Term',
       'Intrrate', 'Crdrate', 'Crdeem', 'Crtsell', 'Creplm', 'OptType',
       'CallOrPut', 'GovInterestRate', 'Spread', 'isPut', 'isCall', 'Post',
       'year'],
      dtype='object')

In [39]:
bnd_info_df.loc[:, const.YEAR] = bnd_info_df['Listdt'].dt.year

In [40]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '财务指标文件.zip'), 'r') as zip_ref:
    with zip_ref.open('CSR_Finidx.csv') as csv_file:
        finidx_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip')
        finidx_df: DataFrame = sort_csmar_data(finidx_df)

In [42]:
finidx_df['ln_at'] = finidx_df['A100000'].apply(np.log)
finidx_df['lev'] = finidx_df['A200000'] / finidx_df['A100000']

In [66]:
# finidx_df[const.YEAR] += 1
bnd_reg_df = bnd_info_df.merge(finidx_df, how='left', left_on=['IssSymbol', const.YEAR], right_on=['tic', const.YEAR])

In [55]:
bnd_reg_df['Liscd'] = bnd_reg_df['Liscd'].astype(int)

In [67]:
reg_df = pd.read_stata(os.path.join(const.OUTPUT_PATH, '20241026_cc_reg_data_v2.dta'))[
    ['tic', const.YEAR, 'has_guarantee']]

bnd_reg_df = bnd_reg_df.drop(['tic'], axis=1).merge(reg_df, how='left', left_on=['IssSymbol', const.YEAR],
                                                    right_on=['tic', const.YEAR])
bnd_reg_df.loc[:, 'has_guarantee'] = bnd_reg_df['has_guarantee'].fillna(0)

In [60]:
bnd_reg_df['rate'] = bnd_reg_df['Crdrate'].apply(lambda x: 0 if pd.isna(x) else 1 if 'B' in x else 2)

In [63]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '债券及主体评级情况表.zip'), 'r') as zip_ref:
    with zip_ref.open('BND_Rating.csv') as csv_file:
        bnd_rate_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip')

bnd_rate_df['DeclareDate'] = pd.to_datetime(bnd_rate_df['DeclareDate'])
bnd_rate_df['Liscd'] = bnd_rate_df['Liscd'].astype(int)

In [64]:
# Merge rating information with bond basic information, using the rating closest to the bond Listdt
def match_rating(corp_row, ratings_df):
    filtered_ratings = ratings_df[ratings_df['Liscd'] == corp_row['Liscd']]
    if filtered_ratings.empty:
        return pd.Series([None, None])

    filtered_ratings['date_diff'] = abs(filtered_ratings['DeclareDate'] - corp_row['Listdt'])
    closest_rating = filtered_ratings.loc[filtered_ratings['date_diff'].idxmin()]
    return pd.Series([closest_rating['BtcrAdj'], closest_rating['CtcrAdj']])

In [65]:
bnd_info_df[['BtcrAdj', 'CtcrAdj']] = bnd_info_df.apply(lambda x: match_rating(x, bnd_rate_df), axis=1)


In [73]:
rating_dict = {'AAA+': 12,
               'AAA': 11,
               'AA+sf': 10,
               'AA+': 10,
               'AA': 9,
               'AA-': 8,
               'AA- ': 8,
               'A+': 7,
               'A': 6,
               'A-1': 5,
               'A-1 ': 5,
               'A-': 5,
               'BBB+': 4,
               'BBB': 3,
               'BBB-': 2,
               'B': 1,
               'C': 0}
for key in ['Crdrate', 'BtcrAdj', 'CtcrAdj']:
    bnd_reg_df[f'{key}_val'] = bnd_reg_df[key].replace(rating_dict)
    bnd_reg_df[key] = bnd_reg_df[key].fillna('C')
    bnd_reg_df[f'{key}_val'] = bnd_reg_df[f'{key}_val'].fillna(0)

C:\Users\wangy\AppData\Local\Temp\ipykernel_20360\566518933.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bnd_reg_df[f'{key}_val'] = bnd_reg_df[key].replace(rating_dict)
C:\Users\wangy\AppData\Local\Temp\ipykernel_20360\566518933.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bnd_reg_df[f'{key}_val'] = bnd_reg_df[key].replace(rating_dict)
C:\Users\wangy\AppData\Local\Temp\ipykernel_20360\566518933.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain t

In [75]:
bnd_reg_df['Liscd'] = bnd_reg_df['Liscd'].astype(int)

In [76]:
bnd_reg_df.to_stata(os.path.join(const.OUTPUT_PATH, '20241027_corporate_bond_spread.dta'), write_index=False,
                    version=119)

# Calculate Investment Efficiency

Source: Baik, D. (Young-I., Chen, C. X., & Godsell, D. (2024). Board Gender Diversity and Investment Efficiency: Global Evidence from 83 Country-Level Interventions. The Accounting Review, 99(3), 1–36. https://doi.org/10.2308/TAR-2022-0251


In [33]:
import statsmodels.api as sm

full_reg_df = pd.read_stata(os.path.join(const.OUTPUT_PATH, '20241026_cc_reg_data.dta'))

reg_df = full_reg_df[['indcd', 'sale_growth', 'tic', 'year', 'CAPEX_RDI_lat_1']].dropna(how='any')

# Add an indicator for negative sales growth
reg_df['negative_sales_growth'] = np.where(reg_df['sale_growth'] < 0, 1, 0)

# Prepare independent variables
# Sales growth and interaction term
reg_df['interaction'] = reg_df['negative_sales_growth'] * reg_df['sale_growth']

# Dependent variable
y = reg_df['CAPEX_RDI_lat_1']

# Estimate the model for each industry code (cross-sectional regression)
residuals = []
for indcd, group in reg_df.groupby('indcd'):
    X = group[['sale_growth', 'negative_sales_growth', 'interaction']]
    X = sm.add_constant(X)  # Adds a constant term for the intercept
    model = sm.OLS(group[y.name], X)
    results = model.fit()
    group['residuals'] = results.resid
    residuals.append(group)

# Combine residuals from all industry groups
df_with_residuals = pd.concat(residuals)

# Calculate investment efficiency
# Absolute value of residual * -1
df_with_residuals['investment_efficiency'] = -1 * df_with_residuals['residuals'].abs()

# Alternate definition: 1 if absolute value of residual * -1 is above median, otherwise 0
median_residual = df_with_residuals['investment_efficiency'].median()
df_with_residuals['investment_efficiency_alt'] = np.where(df_with_residuals['investment_efficiency'] > median_residual,
                                                          1, 0)

# Display the result
df_with_residuals[['indcd', 'CAPEX_RDI_lat_1', 'sale_growth', 'negative_sales_growth', 'investment_efficiency',
                   'investment_efficiency_alt']].head()


,indcd,CAPEX_RDI_lat_1,sale_growth,negative_sales_growth,investment_efficiency,investment_efficiency_alt
16881,,0.002151,0.364444,0,-0.04885,0
16882,,0.006760,0.054600,0,-0.043109,0
16883,,0.001904,0.066165,0,-0.048008,0
16884,,0.001213,0.013319,0,-0.048505,0
16885,,0.000509,0.097705,0,-0.049518,0


In [85]:
df_with_residuals.head()

,indcd,sale_growth,tic,year,CAPEX_RDI_lat_1,negative_sales_growth,interaction,residuals,investment_efficiency,investment_efficiency_alt
16794,,0.364444,200002,2007.0,0.002151,0,0.0,-0.04885,-0.04885,0
16795,,0.054600,200002,2008.0,0.006760,0,0.0,-0.043109,-0.043109,0
16796,,0.066165,200002,2009.0,0.001904,0,0.0,-0.048008,-0.048008,0
16797,,0.013319,200002,2010.0,0.001213,0,0.0,-0.048505,-0.048505,0
16798,,0.097705,200002,2011.0,0.000509,0,0.0,-0.049518,-0.049518,0


In [34]:
full_reg_df2 = full_reg_df.merge(
    df_with_residuals[[const.TICKER, const.YEAR, 'investment_efficiency', 'investment_efficiency_alt']],
    on=[const.TICKER, const.YEAR], how='left')
full_reg_df2['investment_efficiency'] = full_reg_df2['investment_efficiency'].astype(float)
full_reg_df2.to_stata(os.path.join(const.OUTPUT_PATH, '20241027_cc_reg_data.dta'), write_index=False, version=119)

In [84]:
df_with_residuals['indcd'].unique()

array(['', 'A01', 'A02', 'A03', 'A04', 'A05', 'B06', 'B07', 'B08', 'B09',
       'B10', 'B11', 'C13', 'C14', 'C15', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29',
       'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38',
       'C39', 'C40', 'C41', 'C42', 'C43', 'D44', 'D45', 'D46', 'E47',
       'E48', 'E49', 'E50', 'F51', 'F52', 'G53', 'G54', 'G55', 'G56',
       'G58', 'G59', 'G60', 'H61', 'H62', 'I63', 'I64', 'I65', 'J66',
       'J67', 'J68', 'J69', 'K70', 'L71', 'L72', 'M73', 'M74', 'M75',
       'N77', 'N78', 'O79', 'O80', 'O81', 'P82', 'Q83', 'R85', 'R86',
       'R87', 'R88', 'S90'], dtype=object)

# Construct Loan Level data

In [28]:
cbl_loan_df: DataFrame = pd.read_excel(os.path.join(const.CSMAR_PATH, '上市公司贷款', 'CBL_Loan.xlsx'), 
                                       usecols=['Stkcd', 'Time', 'Bank', 'Money', 'Term', 'Starttime', 'Endtime', 'Guarantee', 'Interest']).rename(
    columns={'Time': 'SignDate', 'Bank': 'LoanBank', 'Money': 'LoanAmount', 'Term': 'LoanTerm', 
             'Starttime': 'LoanStartDate', 'Endtime': 'LoanEndDate', 'Guarantee': 'IsPledge', 'Interest': 'IntersetRate'})
cbl_loan_df = cbl_loan_df.iloc[2:].dropna(subset=['IntersetRate'], how='any')

In [36]:
import re

# Function to extract the main bank name
def extract_main_bank_name(bank_name):
    # Match the main bank name before any branch identifiers like '支行', '分行', or '营业部'
    match = re.match(r'^(.*?银行)', bank_name)
    return match.group(1) if match else bank_name

cbl_loan_df.loc[797, 'LoanStartDate'] = '2008-08-01'
cbl_loan_df.loc[797, 'LoanEndDate'] = '2015-07-31'
cbl_loan_df.loc[11726, 'LoanStartDate'] = '2006-08-01'
cbl_loan_df.loc[11726, 'LoanEndDate'] = '2009-07-31'
cbl_loan_df.loc[12839, 'LoanStartDate'] = '2005-06-01'
cbl_loan_df.loc[12839, 'LoanEndDate'] = '2006-05-31'
cbl_loan_df.loc[12840, 'LoanStartDate'] = '2005-03-01'
cbl_loan_df.loc[12840, 'LoanEndDate'] = '2006-02-28'
cbl_loan_df['IsPledge'] = cbl_loan_df['IsPledge'].notnull().astype(int)
cbl_loan_df['LoanBank'] = cbl_loan_df['LoanBank'].apply(extract_main_bank_name)
cbl_loan_df['LoanStartDate'] = pd.to_datetime(cbl_loan_df['LoanStartDate'])
cbl_loan_df['LoanEndDate'] = pd.to_datetime(cbl_loan_df['LoanEndDate'])
cbl_loan_df['LoanTerm'] = cbl_loan_df['LoanTerm'].fillna((cbl_loan_df['LoanEndDate'] - cbl_loan_df['LoanStartDate']).dt.days / 365)


In [38]:
cbl_loan_df.to_pickle(os.path.join(const.TEMP_PATH, 'pre2013_cbl_loan_df.pkl'))

In [4]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '上市公司贷款', '2013_2017上市公司向银行借款表(日).zip'), 'r') as zip_ref:
    with zip_ref.open('CBL_LoanNew.csv') as csv_file:
        cbl_new_loan_df1: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip',
                                           usecols=['Symbol', 'DeclareDate', 'LoanBank', 'SignDate', 'MaxLoan', 'LoanTerm', 'LoanStarDate', 
                                                    'LoanEndDate', 'BaseRate', 'IntersetRate', 'IsPledge']).rename(
            columns={'Symbol': 'Stkcd'}).dropna(subset=['IntersetRate'], how='all')
        
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '上市公司贷款', '2018_2019上市公司向银行借款表(日).zip'), 'r') as zip_ref:
    with zip_ref.open('CBL_LoanNew.csv') as csv_file:
        cbl_new_loan_df2: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip',
                                           usecols=['Symbol', 'DeclareDate', 'LoanBank', 'SignDate', 'MaxLoan', 'LoanTerm', 'LoanStarDate', 
                                                    'LoanEndDate', 'BaseRate', 'IntersetRate', 'IsPledge']).rename(
            columns={'Symbol': 'Stkcd'}).dropna(subset=['IntersetRate'], how='all')

cbl_new_loan_df: DataFrame = pd.concat([cbl_new_loan_df1, cbl_new_loan_df2], ignore_index=True)

C:\Users\wangy\AppData\Local\Temp\ipykernel_1112\4103463007.py:3: DtypeWarning: Columns (17,18,19,20,24) have mixed types. Specify dtype option on import or set low_memory=False.
  cbl_new_loan_df1: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip',
C:\Users\wangy\AppData\Local\Temp\ipykernel_1112\4103463007.py:11: DtypeWarning: Columns (14,17,18,19,20,24,27,28,32,33,36) have mixed types. Specify dtype option on import or set low_memory=False.
  cbl_new_loan_df2: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip',


In [5]:
cbl_new_loan_df.head()

,Stkcd,DeclareDate,LoanBank,SignDate,MaxLoan,MinLoan,LatestLoanAmount,AccumLoanAmount,LoanTerm,LoanStarDate,LoanEndDate,BaseRate,IntersetRate,IsPledge
0,000002,2014-04-22,华润深国投信托有限公司,NaN,75000.0,75000.0,NaN,NaN,2.0,NaN,NaN,6.15,7.482,0.0
1,000005,2016-09-06,上海浦东发展银行股份有限公司,2016-09-02,6000.0,6000.0,NaN,NaN,1.0,NaN,NaN,4.35,4.35,0.0
2,000005,2017-04-22,长安国际信托股份有限公司,2017-04-20,10000.0,10000.0,NaN,NaN,1.0,NaN,NaN,4.35,7.0,0.0
3,000030,2015-04-29,一汽财务有限公司,NaN,20000.0,0.0,NaN,NaN,1.0,NaN,NaN,5.35,3.5,0.0
4,000030,2016-04-21,一汽财务有限公司,NaN,12000.0,0.0,NaN,NaN,1.0,NaN,NaN,4.35,2.0,0.0


In [22]:
cbl_loan_df2: DataFrame = cbl_new_loan_df.drop([1536, 1596, 1608])
cbl_loan_df2['SignDate'] = pd.to_datetime(cbl_loan_df2['SignDate'].fillna(cbl_loan_df2['DeclareDate']), errors='coerce')
cbl_loan_df2['LoanAmount'] = cbl_loan_df2['MaxLoan']
cbl_loan_df2.dropna(subset=['SignDate'], how='any', inplace=True)
cbl_loan_df2.dropna(subset=['LoanTerm', 'LoanStarDate', 'LoanEndDate'], how='all', inplace=True)
cbl_loan_df2['LoanEndDate'] = pd.to_datetime(cbl_loan_df2['LoanEndDate'])
cbl_loan_df2['LoanTerm'] = cbl_loan_df2['LoanTerm'].fillna((cbl_loan_df2['LoanEndDate'] - cbl_loan_df2['SignDate']).dt.days / 365)

In [ ]:
key_to_drop = ['MaxLoan', 'MinLoan', 'LatestLoanAmount', 'AccumLoanAmount']

# Append rating data

In [3]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '上市公司贷款', '上市公司信用评级情况表.zip'), 'r') as zip_ref:
    with zip_ref.open('DEBT_BOND_RATING.csv') as csv_file:
        rating_df: DataFrame = pd.read_csv(csv_file, on_bad_lines='skip').dropna(subset=['LongTermRating'])


In [4]:
rating_df['RatingDate'] = pd.to_datetime(rating_df['RatingDate'])

In [5]:
rating_df[const.YEAR] = rating_df['RatingDate'].dt.year
rating_df['Prospect'] = rating_df['RatingProspect'].replace({'稳定': 0, '正面': 1, '负面': -1, '待决': np.nan})

/tmp/ipykernel_15040/3367601164.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rating_df['Prospect'] = rating_df['RatingProspect'].replace({'稳定': 0, '正面': 1, '负面': -1, '待决': np.nan})


In [6]:
import datetime
rating_df['Post'] = (rating_df['RatingDate'] > datetime.datetime(2014, 5, 19)).astype(int)

In [7]:
rating_mapping = {
    'AAA+': 13,
    'AAA': 12,
    'AAA-': 11,
    'AA+': 10,
    'AA': 9,
    'AA-': 8,
    'A+': 7,
    'A': 6,
    'A-1': 5,
    'A-': 5,
    'BBB+': 4,
    'BBB': 4,
    'BBB-': 4,
    'BB+': 3,
    'BB': 3,
    'BB-': 3,
    'B+': 2,
    'B': 2,
    'B-': 2,
    'CCC': 1,
    'CC': 1,
    'C': 1
}

rating_mapping7 = {
    'AAA+': 7,
    'AAA': 7,
    'AAA-': 7,
    'AA+': 6,
    'AA': 5,
    'AA-': 4,
    'A+': 3,
    'A': 2,
    'A-1': 2,
    'A-': 2,
    'BBB+': 1,
    'BBB': 1,
    'BBB-': 1,
    'BB+': 1,
    'BB': 1,
    'BB-': 1,
    'B+': 1,
    'B': 1,
    'B-': 1,
    'CCC': 1,
    'CC': 1,
    'C': 1
}

# [1]常莹莹,曾泉.环境信息透明度与企业信用评级——基于债券评级市场的经验证据[J].金融研究,2019,(05):132-151.
rating_mapping5 = {
    'AAA+': 5,
    'AAA': 5,
    'AAA-': 5,
    'AA+': 4,
    'AA': 4,
    'AA-': 4,
    'A+': 3,
    'A': 3,
    'A-1': 3,
    'A-': 3,
    'BBB+': 2,
    'BBB': 2,
    'BBB-': 2,
    'BB+': 1,
    'BB': 1,
    'BB-': 1,
    'B+': 1,
    'B': 1,
    'B-': 1,
    'CCC': 0,
    'CC': 0,
    'C': 0
}

In [8]:
rating_df.loc[:, 'RatingScore13'] = rating_df['LongTermRating'].replace(rating_mapping)
rating_df.loc[:, 'RatingScore7'] = rating_df['LongTermRating'].replace(rating_mapping7)
rating_df.loc[:, 'RatingScore5'] = rating_df['LongTermRating'].replace(rating_mapping5)

/tmp/ipykernel_15040/2637504357.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rating_df.loc[:, 'RatingScore13'] = rating_df['LongTermRating'].replace(rating_mapping)
/tmp/ipykernel_15040/2637504357.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rating_df.loc[:, 'RatingScore7'] = rating_df['LongTermRating'].replace(rating_mapping7)
/tmp/ipykernel_15040/2637504357.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `resul

In [86]:
useless_id = {10486199, 10904, 104724, 105457, 10123793, 10251032, 10486104, 10486199}

In [16]:
rating_df[['RatingInstitution', 'RatingInstitutionID']].drop_duplicates()

,RatingInstitution,RatingInstitutionID
0,联合资信评估有限公司,10897
3,中诚信国际信用评级有限责任公司,105282
10,大公国际资信评估有限公司,10898
54,中诚信证券评估有限公司,10905
120,联合资信评估股份有限公司,10897
148,鹏元资信评估有限公司,10903
163,联合信用评级有限公司,10906
171,上海新世纪资信评估投资服务有限公司,10900
214,中债资信评估有限责任公司,10108920
287,上海远东资信评估有限公司,10904


In [84]:
valid_rating_df = rating_df.copy()
valid_rating_df.groupby('RatingInstitutionID')['Symbol'].count()

RatingInstitutionID
10897       3639
10898       1385
10900       2711
10903       1935
10904         39
10905       1748
10906       1718
104724         1
105282      5344
105457         1
10102298     823
10108920     127
10123793       3
10251032       2
10486104       2
10486199       7
Name: Symbol, dtype: int64

In [71]:
valid_rating_df['RatingScore'] = pd.to_numeric(valid_rating_df['RatingScore'], errors='coerce')

In [92]:
mean_rating = valid_rating_df.loc[~valid_rating_df['RatingInstitutionID'].isin(useless_id)].groupby(
    ['Symbol', 'year'])[['RatingScore', 'Prospect']].mean()
rating_count = valid_rating_df.loc[~valid_rating_df['RatingInstitutionID'].isin(useless_id)].groupby(
    ['Symbol', 'year'])[['RatingInstitutionID']].count()
mean_rating = mean_rating.merge(rating_count, left_index=True, right_index=True)

In [93]:

all_years = pd.MultiIndex.from_product([mean_rating.index.levels[0], range(2008, 2021)], names=['Symbol', 'year'])
mean_rating = mean_rating.reindex(all_years)
mean_rating.ffill(inplace=True)
mean_rating.reset_index(inplace=True)
mean_rating


,Symbol,year,RatingScore,Prospect,RatingInstitutionID
0,1,2008,18.0,0.0,2.0
1,1,2009,19.0,0.0,2.0
2,1,2010,19.0,0.0,2.0
3,1,2011,19.0,0.0,4.0
4,1,2012,20.0,0.0,5.0
...,...,...,...,...,...
22485,900950,2016,17.5,0.0,2.0
22486,900950,2017,20.0,0.0,1.0
22487,900950,2018,20.0,0.0,1.0
22488,900950,2019,20.0,0.0,1.0


In [94]:
mean_rating['rating_diff'] = mean_rating.groupby('Symbol')['RatingScore'].diff()
mean_rating['is_raise'] = (mean_rating['rating_diff'] > 0).astype(int)
mean_rating['is_decrease'] = (mean_rating['rating_diff'] < 0).astype(int)


In [82]:
mean_rating[const.YEAR] += 2

In [95]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.OUTPUT_PATH, '20241027_cc_reg_data.dta'))
mean_rating.rename(columns={'Symbol': const.TICKER}, inplace=True)
reg_df2 = reg_df.merge(mean_rating, on=[const.TICKER, const.YEAR], how='left')
mean_rating[const.YEAR] -= 1

reg_df2 = reg_df2.merge(mean_rating, on=[const.TICKER, const.YEAR], how='left', suffixes=('', '_1'))
reg_df2.to_stata(os.path.join(const.OUTPUT_PATH, '202411011_cc_reg_data.dta'), write_index=False, version=119)

In [9]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.OUTPUT_PATH, '20241027_cc_reg_data.dta'))
rating_df[const.YEAR] = rating_df['RatingDate'].dt.year
rating_df['RatingScore21'] = rating_df['RatingScore13'].astype(float)
rating_df['RatingScore7'] = rating_df['RatingScore7'].astype(float)
rating_df['RatingScore5'] = rating_df['RatingScore5'].astype(float)
tmp_reg_df = reg_df.copy()
tmp_reg_df[const.YEAR] -= 1
rating_reg_df = rating_df.rename(columns={'Symbol': const.TICKER}).merge(tmp_reg_df, on=[const.TICKER, const.YEAR], how='left').merge(
    reg_df, on=[const.TICKER, const.YEAR], how='left', suffixes=("", "_c"))
rating_reg_df.to_stata(os.path.join(const.OUTPUT_PATH, '20241101_cc_rating_reg_data.dta'), write_index=False, version=119)

In [19]:
rating_reg_df.groupby('RatingInstitutionID')['has_guarantee_c'].sum()

RatingInstitutionID
10897       324.0
10898       174.0
10900       258.0
10903        82.0
10904         1.0
10905       375.0
10906       336.0
104724        0.0
105282      582.0
105457        0.0
10102298     75.0
10108920     15.0
10123793      0.0
10251032      0.0
10486104      0.0
10486199      0.0
Name: has_guarantee_c, dtype: float64

In [21]:
rating_reg_df.groupby('RatingInstitutionID')['has_guarantee'].sum()

RatingInstitutionID
10897       207.0
10898       149.0
10900       234.0
10903        89.0
10904         1.0
10905       382.0
10906       249.0
104724        0.0
105282      423.0
105457        0.0
10102298     56.0
10108920     18.0
10123793      0.0
10251032      0.0
10486104      0.0
10486199      0.0
Name: has_guarantee, dtype: float64

In [22]:
rating_reg_df.groupby('RatingInstitutionID')['has_guarantee'].count()

RatingInstitutionID
10897       1127
10898        932
10900       1527
10903        978
10904         17
10905       1660
10906       1338
104724         0
105282      2071
105457         1
10102298     281
10108920      92
10123793       0
10251032       0
10486104       0
10486199       0
Name: has_guarantee, dtype: int64

In [35]:
rating_reg_df2 = rating_reg_df.loc[rating_reg_df['RatingInstitutionID'].isin(
    {10897, 10898, 10900, 10903, 10905, 10906, 105282, 10102298, 10108920})]
reting_reg_df3 = rating_reg_df2.loc[:, ['tic', 'DeclareDate', 'RatingDate', 'RatingInstitutionID', 
                                        'LongTermRating', 'RatingProspect', 'RatingInstitution',
                                        'year', 'Post', 'RatingScore13', 'RatingScore7', 'RatingScore5', 'has_guarantee', 
                                        'NumGuarantee']].copy()

In [36]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, 'CSMAR_06-20.zip'), 'r') as zip_ref:
    with zip_ref.open('CSMAR_06-20.dta') as dta_file:
        finidx_df: DataFrame = pd.read_stata(dta_file).rename(
            columns={'code': const.TICKER}).drop(['businessscope', 'mainbussiness', '_merge', 'sigchange',
                                                   'shortname', 'indcd', 'prci', 'city', 'accper', 'typrep', 
                                                  'reptdt', 'annodt', 'y0801b', 'y0901b', 'y1901b', 'enddate', 
                                                  'largestholder', 'actualcontrollername', 'actualcontrollernatureid', 
                                                  'sharesnature', 'equitynature', 'equitynatureid', 'hierarchy', 
                                                  'founder', 'originator', 'auditdate', 'typeauditopin', 'auditor', 
                                                  'territoryaccountingfirm', 'outsideaccountingfirm', 'territoryauditfeecurrency', 
                                                  'outsideauditfeecurrency', 'otherfeecurrencycode', 'totalcurrencycode', 
                                                  'nonstandexplai', 'auditfeedirec', 'industryname', 'stknmec', 'companyopacity', 
                                                  'csr等级', 'csr技术创新理念', 'csr员工培训', 'csr安全检查', 'csr安全培训', 
                                                  'csr慰问意识', 'csr慰问人', 'csr慰问金', 'csr质量管理意识', 'csr质量管理体系证书', 
                                                  'csr客户满意度调查', 'csr供应商公平竞争', 'csr反商业贿赂培训', 'csr环保意识', 
                                                  'csr环境管理体系认证', 'zipcode', 'secretary', 'secretarytel', 
                                                  'secretaryfax', 'secretaryemail', 'securityconsultant', 
                                                  'socialcreditcode', 'sigchange', 'isin', 'fullname', 
                                                  'legalrepresentative', 'establishdate', 'crcd', 'website', 
                                                  'email', 'listingdate', 'province', 'listingstate', 'coname', 
                                                  'coname_cn', 'prvn', 'pftn', 'cont', 'indusa', 'indcodea', 'indusb', 
                                                  'indcodeb', 'offadd', 'officeaddress', 'registeraddress', 'regstadd'], axis=1)

reting_reg_df4: DataFrame = reting_reg_df3.merge(finidx_df, on=[const.TICKER, const.YEAR], how='left')
finidx_df[const.YEAR] -= 1
reting_reg_df4: DataFrame = reting_reg_df4.merge(finidx_df, on=[const.TICKER, const.YEAR], how='left', suffixes=('', '_l'))
for key in ['has_guarantee', 'NumGuarantee']:
    reting_reg_df4.loc[:, key] = reting_reg_df4[key].fillna(0)

In [25]:
print(reting_reg_df4.dtypes)

tic                             int64
DeclareDate                    object
RatingDate             datetime64[ns]
RatingInstitutionID             int64
LongTermRating                 object
                            ...      
seonetvalue_l                 float32
seo_l                         float32
seoratio_l                    float32
seonetratio_l                 float32
dividend_l                    float32
Length: 1663, dtype: object


In [33]:
reting_reg_df4.shape

(19430, 1509)

In [37]:
reting_reg_df4.to_stata(os.path.join(const.OUTPUT_PATH, '20241101_cc_rating_reg_data_v2.dta'), 
                        write_index=False, version=119)

In [27]:
reting_reg_df4[col].dtype

dtype('float32')

In [31]:
# Find columns with unsupported data types (e.g., object type but not pure strings)
unsupported_columns = []

for col in reting_reg_df4.columns:
    # Check if the column has unsupported data types
    if reting_reg_df4[col].dtype == object:
        unsupported_columns.append(col)

# Display the columns with unsupported data types or mixed content in string columns
print("Columns with unsupported data types or mixed types:", unsupported_columns)

Columns with unsupported data types or mixed types: ['DeclareDate', 'LongTermRating', 'RatingProspect', 'RatingInstitution', 'shortname', 'indcd', 'industrycode', 'prci', 'city', 'accper', 'typrep', 'reptdt', 'annodt', 'y0801b', 'y0901b', 'y1901b', 'enddate', 'largestholder', 'actualcontrollername', 'actualcontrollernatureid', 'sharesnature', 'equitynature', 'equitynatureid', 'hierarchy', 'founder', 'originator', 'auditdate', 'typeauditopin', 'auditor', 'territoryaccountingfirm', 'outsideaccountingfirm', 'territoryauditfeecurrency', 'outsideauditfeecurrency', 'otherfeecurrencycode', 'totalcurrencycode', 'nonstandexplai', 'auditfeedirec', 'industryname', 'stknmec', 'companyopacity', 'csr等级', 'csr技术创新理念', 'csr员工培训', 'csr安全检查', 'csr安全培训', 'csr慰问意识', 'csr慰问人', 'csr慰问金', 'csr质量管理意识', 'csr质量管理体系证书', 'csr客户满意度调查', 'csr供应商公平竞争', 'csr反商业贿赂培训', 'csr环保意识', 'csr环境管理体系认证', 'zipcode', 'secretary', 'secretarytel', 'secretaryfax', 'secretaryemail', 'securityconsultant', 'socialcreditcode', 'sigchange'

# Merge Foreign Related data
## step 1 operation revenue

In [3]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '海外直接投资', '海外业务收入表.zip'), 'r') as zip_ref:
    with zip_ref.open('OFDI_OPERATEINCOME.xlsx') as excel_file:
        ofdi_opeartion_df: DataFrame = pd.read_excel(excel_file)

D:\Users\wangy\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
ofdi_opeartion_df = ofdi_opeartion_df.iloc[2:]
ofdi_opeartion_df['EndDate'] = pd.to_datetime(ofdi_opeartion_df['EndDate'])

In [6]:
ofdi_opeartion_df[const.YEAR] = ofdi_opeartion_df['EndDate'].dt.year
ofdi_opeartion_df[const.TICKER] = ofdi_opeartion_df['Symbol'].astype(int)

In [17]:
for key in ['Earnings', 'EarningsProportion', 'Costs', 'GrossMargin', 'GrossMarginProportion', 'GrossMarginRate']:
    ofdi_opeartion_df[key] = pd.to_numeric(ofdi_opeartion_df[key])

In [18]:
earning_cost = ofdi_opeartion_df.groupby([const.TICKER, const.YEAR])[['Earnings', 'EarningsProportion', 'Costs', 
                                                                      'GrossMargin', 'GrossMarginProportion']].sum()
profit_margin = ofdi_opeartion_df.groupby([const.TICKER, const.YEAR])[['GrossMarginRate']].mean()
ofdi_op_df: DataFrame = earning_cost.merge(profit_margin, left_index=True, right_index=True).reset_index(drop=False)

In [22]:
for key in ['Earnings', 'EarningsProportion', 'Costs', 'GrossMargin', 'GrossMarginProportion', 'GrossMarginRate']:
    ofdi_op_df.loc[ofdi_op_df[key].notnull(), key] = winsorize(ofdi_op_df[key].dropna(), limits=(0.005, 0.005))

SyntaxError: invalid syntax (202694357.py, line 2)

In [23]:
ofdi_op_df.describe()

,tic,year,Earnings,EarningsProportion,Costs,GrossMargin,GrossMarginProportion,GrossMarginRate
count,27234.000000,27234.000000,2.723400e+04,27234.000000,2.723400e+04,2.723400e+04,27234.000000,18350.000000
mean,303135.966916,2016.994933,3.171992e+09,54.441355,1.529349e+09,3.220780e+08,32.761380,25.376294
std,274391.891038,4.821072,1.024779e+10,67.633947,5.846688e+09,1.150797e+09,61.410259,20.609258
min,1.000000,2003.000000,1.221584e+05,0.003600,0.000000e+00,-8.890504e+07,-48.879800,-66.280800
25%,2365.250000,2014.000000,1.127073e+08,7.169475,0.000000e+00,0.000000e+00,0.000000,12.652579
50%,300315.000000,2018.000000,5.187112e+08,28.546950,8.299729e+07,2.232888e+07,5.514950,22.267067
75%,600601.000000,2021.000000,1.841511e+09,76.298900,6.569451e+08,1.748629e+08,39.641875,34.938875
max,900956.000000,2023.000000,9.623320e+10,359.411000,5.645614e+10,1.153920e+10,365.204300,97.946500


## append share structure

In [25]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '海外直接投资', '股权信息表.zip'), 'r') as zip_ref:
    with zip_ref.open('OFDI_LISTSTKRIGHT.xlsx') as excel_file:
        ofdi_ownership_df: DataFrame = pd.read_excel(excel_file)

D:\Users\wangy\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [28]:
ofdi_ownership_df.head()

,Symbol,ShortName,EndDate,EnterpriseAge,TotNumShares,NonTradableShares,LockShares,TradeShares,Zindex,Herfindahl5,Herfindahl10,InsInvestorProp,StatesharesProp,ForeignInvestorProp,BalanceIndicators,TotalNumSectoFifth,ShareholderFirstProp
0,证券代码,证券简称,统计截止日期,企业年龄,总股数,未流通股股数,限售流通股股数,已流通股股数,Z指数,Herfindahl_5指数,Herfindahl_10指数,机构投资者持股比例,国有股持股比例,境外投资者持股比例,股权制衡指标,其中：第2-5大股东持股比例合计,其中：第一大股东持股比例
1,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位
2,000001,平安银行,2003-12-31,16,1945822149,536460184,0,1409361965,1.0653,0.000058,0.000066,26.6754,7.08,NaN,2.0805,14.73,7.08
3,000001,平安银行,2004-12-31,17,1945822149,536460184,NaN,1409361965,1.0111,0.000059,0.000082,27.8712,4,17.89,0.3924,7.02,17.89
4,000001,平安银行,2005-12-31,18,1945822149,536460184,NaN,1409361965,1.7563,0.000121,0.000153,29.5501,3.2,17.89,0.393,7.03,17.89


In [31]:
# ofdi_ownership_df = ofdi_ownership_df.iloc[2:]
ofdi_ownership_df['EndDate'] = pd.to_datetime(ofdi_ownership_df['EndDate'])
ofdi_ownership_df[const.YEAR] = ofdi_ownership_df['EndDate'].dt.year
ofdi_ownership_df[const.TICKER] = ofdi_ownership_df['Symbol'].astype(int)
ofdi_ownership_df['ForeignInvestorProp'] = pd.to_numeric(ofdi_ownership_df['ForeignInvestorProp'])
ofdi_ownership_df['StatesharesProp'] = pd.to_numeric(ofdi_ownership_df['StatesharesProp'])
ofdi_ownership_df['InsInvestorProp'] = pd.to_numeric(ofdi_ownership_df['InsInvestorProp'])

C:\Users\wangy\AppData\Local\Temp\ipykernel_26204\1618070116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofdi_ownership_df['EndDate'] = pd.to_datetime(ofdi_ownership_df['EndDate'])
C:\Users\wangy\AppData\Local\Temp\ipykernel_26204\1618070116.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofdi_ownership_df[const.YEAR] = ofdi_ownership_df['EndDate'].dt.year
C:\Users\wangy\AppData\Local\Temp\ipykernel_26204\1618070116.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [42]:
ofdi_df: DataFrame = ofdi_op_df.merge(ofdi_ownership_df[[const.TICKER, const.YEAR, 'InsInvestorProp', 'StatesharesProp', 'ForeignInvestorProp']], 
                                      on=[const.TICKER, const.YEAR], how='outer')

## Count foreign firms number

In [34]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '海外直接投资', '海外关联公司表.zip'), 'r') as zip_ref:
    with zip_ref.open('OFDI_AFFCOMPINFO.xlsx') as excel_file:
        ofdi_subfirm_df: DataFrame = pd.read_excel(excel_file).iloc[2:]

D:\Users\wangy\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [35]:
ofdi_subfirm_df.head()

,Symbol,ShortName,EndDate,RalatedParty,RelationshipID,RegisterCapital,BusinessScope,RegisterAddress,CountryName,IsBRCountry,...,DirectHoldingRatio,IndirectHoldingRatio,TotalHoldingRatio,TotalAssets,OperatingEvenue,NetProfit,TotalCost,CorporateIncomeTax,IsExit,Currency
2,000002,万科A,1999-12-31,永达中国投资有限公司,1,10000,投资,香港,中国香港,0,...,100,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,注册资本HKD；其他CNY
3,000002,万科A,1999-12-31,万科影视有限公司,1,50000,投资,香港,中国香港,0,...,NaN,100,100,NaN,NaN,NaN,NaN,NaN,NaN,注册资本HKD；其他CNY
4,000002,万科A,1999-12-31,万科中国投资有限公司,1,50000,投资,香港,中国香港,0,...,NaN,100,100,NaN,NaN,NaN,NaN,NaN,NaN,注册资本HKD；其他CNY
5,000002,万科A,1999-12-31,万科企业股份有限公司美国公司,1,100000,商贸,美国,美国,0,...,100,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,注册资本USD；其他CNY
6,000002,万科A,2000-12-31,永达中国投资有限公司,1,10000,投资,NaN,中国香港,0,...,100,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,注册资本HKD；其他CNY


In [37]:
ofdi_subfirm_df[const.TICKER] = ofdi_subfirm_df['Symbol'].astype(int)
ofdi_subfirm_df['EndDate'] = pd.to_datetime(ofdi_subfirm_df['EndDate'])
ofdi_subfirm_df[const.YEAR] = ofdi_subfirm_df['EndDate'].dt.year


In [40]:
for key in ['RegisterCapital', 'IsBRCountry', 'IsTaxHaven', 'TotalAssets', 'OperatingEvenue', 'NetProfit', 'TotalCost']:
    ofdi_subfirm_df[key] = pd.to_numeric(ofdi_subfirm_df[key])
    
num_subfirm_df = ofdi_subfirm_df.groupby([const.TICKER, const.YEAR])['Symbol'].count()
sub_firm_amount = ofdi_subfirm_df.groupby([const.TICKER, const.YEAR])[[
    'RegisterCapital', 'IsBRCountry', 'IsTaxHaven', 'TotalAssets', 'OperatingEvenue', 'NetProfit', 'TotalCost']].sum()
ofdi_subfirm_df: DataFrame = sub_firm_amount.merge(num_subfirm_df, left_index=True, right_index=True).reset_index(
    drop=False).rename(columns={'Symbol': 'SubFirmNum', 'RegisterCapital': 'SubFirmAmount', 'IsBRCountry': 'BRCountryNum',
                                'IsTaxHaven': 'TaxHavenNum', 'TotalAssets': 'SubFirmAT', 'OperatingEvenue': 'SubFirmOE', 'NetProfit': 'SubFirmProfit',
                                'TotalCost': 'SubFirmInvest'})

In [43]:
ofdi_df: DataFrame = ofdi_df.merge(ofdi_subfirm_df, on=[const.TICKER, const.YEAR], how='outer')

In [46]:
ofdi_df.rename(columns={'Earnings': 'ForeignEarnings', 'EarningsProportion': 'ForeignEarningsProportion', 'Costs': 'ForeignCosts', 
                        'GrossMargin': 'ForeignMargin', 'GrossMarginRate': 'ForeignMarginRate', 'GrossMarginProportion': 'ForeignMarginRatio'}, inplace=True)

In [47]:
ofdi_df.head()

,tic,year,ForeignEarnings,ForeignEarningsProportion,ForeignCosts,ForeignMargin,ForeignMarginRatio,ForeignMarginRate,InsInvestorProp,StatesharesProp,ForeignInvestorProp,SubFirmAmount,BRCountryNum,TaxHavenNum,SubFirmAT,SubFirmOE,SubFirmProfit,SubFirmInvest,SubFirmNum
0,1,2003,NaN,NaN,NaN,NaN,NaN,NaN,26.6754,7.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2004,NaN,NaN,NaN,NaN,NaN,NaN,27.8712,4.00,17.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2005,NaN,NaN,NaN,NaN,NaN,NaN,29.5501,3.20,17.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2006,NaN,NaN,NaN,NaN,NaN,NaN,38.6432,3.20,17.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2007,NaN,NaN,NaN,NaN,NaN,NaN,61.7833,2.99,16.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Construct MA information

In [59]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '海外直接投资', '交易信息总表.zip'), 'r') as zip_ref:
    with zip_ref.open('OFDI_TRADINGMAIN.xlsx') as excel_file:
        ofdi_ma_df: DataFrame = pd.read_excel(excel_file).iloc[2:]

D:\Users\wangy\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [49]:
ofdi_ma_df.head()

,EventID,Symbol,ShortName,FirstDeclareDate,Buyer,Seller,Underlying,LatestDeclareDate,FinishDeclareDate,LastSchedule,...,IsSucceed,RestructuringTypeID,UnderlyingTypeID,UnderlyingValue,ExpenseValue,PayTypeID,SourceTypesID,RelevanceSign,MajorRestructuringSign,Outline
2,260000000001,000001,平安银行,2004-06-01,"Newbridge Asia AIVⅢ,L.P.","深圳国际信托投资有限责任公司,深圳市城市建设开发(集团)公司,深圳市劳动和社会保障局,深圳市...",深圳发展银行股份有限公司,2004-12-30,2004-12-31,完成或者过户,...,Y,S3008,S3202,NaN,1235471061,Q1602,Q2102,N,N,"NewbridgeAsiaAIVⅢ,L.P.已于2004年5月29日与深圳市投资管理公司、深..."
3,260000000006,000001,平安银行,2009-06-16,中国平安保险(集团)股份有限公司,"Newbridge Asia AIV III, L.P.",深圳发展银行股份有限公司,2010-05-08,2010-05-08,完成或者过户,...,Y,S3008,S3202,NaN,11449117658,Q1602,Q2102,N,N,"2009年6月12日,信息披露义务人与中国平安签署了《股份购买协议》,协议主要内容如下:转让..."
4,260000000008,000002,万科A,2000-06-22,FULL KNOWLEDGE INVESTMENTS LIMITED,深圳经济特区发展(集团)公司,万科企业股份有限公司,2000-06-20,2000-06-22,取消方案,...,N,S3008,S3202,NaN,228153971.54,Q1602,Q2102,N,N,"本公司接获第一大股东深圳经济特区发展(集团)公司(“特发公司”)及中国华润总公司知会,双方于..."
5,210000002316,000002,万科A,2005-12-20,Reco Ziyang Pte Ltd.,永达中国投资有限公司,沈阳万科永达房地产开发有限公司,2005-12-20,NaN,董事会通过,...,NaN,S3002,S3202,NaN,99188370,Q1602,Q2102,N,N,万科A董事会决议将公司之全资附属公司永达中国投资有限公司持有的沈阳万科永达房地产开发有限公司...
6,210000002317,000002,万科A,2005-12-20,Reco Ziyang Pte Ltd.,上海万科房地产集团有限公司,无锡万科房地产有限公司,2005-12-20,NaN,董事会通过,...,NaN,S3002,S3202,NaN,192600000,Q1602,Q2102,N,N,万科A董事会决议将公司之全资附属公司上海万科房地产集团有限公司持有的无锡万科房地产有限公司4...


In [60]:
ofdi_ma_df[const.TICKER] = ofdi_ma_df.Symbol.astype(int)
ofdi_ma_df['FinishDeclareDate'] = pd.to_datetime(ofdi_ma_df['FinishDeclareDate'])
ofdi_ma_df[const.YEAR] = ofdi_ma_df['FinishDeclareDate'].dt.year
ofdi_ma_df['ExpenseValue'] = pd.to_numeric(ofdi_ma_df['ExpenseValue'])
ofdi_ma_df = ofdi_ma_df.loc[ofdi_ma_df['IsSucceed'] != 'N']

In [61]:
ma_num = ofdi_ma_df.groupby([const.TICKER, const.YEAR])['EventID'].count()
ma_amt = ofdi_ma_df.groupby([const.TICKER, const.YEAR])['ExpenseValue'].sum()
ofdi_foreign_ma = pd.concat([ma_amt, ma_num], axis=1)

In [63]:
ofdi_fma_df = ofdi_foreign_ma.reset_index(drop=False).rename(columns={'ExpenseValue': 'ForeignMAValue', 'EventID': 'ForeignMANum'})

In [64]:
ofdi_df = ofdi_df.merge(ofdi_fma_df, on=[const.TICKER, const.YEAR], how='left')

## append basic financial information

In [66]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '海外直接投资', '财务指标表.zip'), 'r') as zip_ref:
    with zip_ref.open('OFDI_FININDEX.xlsx') as excel_file:
        finindex_df: DataFrame = pd.read_excel(excel_file, usecols=['Symbol', 'EndDate', 'TotalAsset']).iloc[2:]
        
finindex_df['EndDate'] = pd.to_datetime(finindex_df['EndDate'])
finindex_df[const.YEAR] = finindex_df['EndDate'].dt.year
finindex_df[const.TICKER] = finindex_df['Symbol'].astype(int)
finindex_df['TotalAsset'] = pd.to_numeric(finindex_df['TotalAsset'])

D:\Users\wangy\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [67]:
tmp_fin_df = finindex_df[[const.TICKER, const.YEAR, 'TotalAsset']].copy()
tmp_fin_df[const.YEAR] -= 1
ofdi_at_df: DataFrame = ofdi_df.merge(finindex_df[[const.TICKER, const.YEAR, 'TotalAsset']], on=[const.TICKER, const.YEAR], how='left').merge(
    tmp_fin_df, on=[const.TICKER, const.YEAR], how='left', suffixes=('', '_1'))

In [68]:
ofdi_at_df.keys()

Index(['tic', 'year', 'ForeignEarnings', 'ForeignEarningsProportion',
       'ForeignCosts', 'ForeignMargin', 'ForeignMarginRatio',
       'ForeignMarginRate', 'InsInvestorProp', 'StatesharesProp',
       'ForeignInvestorProp', 'SubFirmAmount', 'BRCountryNum', 'TaxHavenNum',
       'SubFirmAT', 'SubFirmOE', 'SubFirmProfit', 'SubFirmInvest',
       'SubFirmNum', 'ForeignMAValue', 'ForeignMANum', 'TotalAsset',
       'TotalAsset_1'],
      dtype='object')

In [69]:
for key in ['ForeignEarnings', 'ForeignCosts', 'ForeignMargin', 'SubFirmAmount', 'SubFirmAT', 'SubFirmOE', 'SubFirmProfit', 'SubFirmInvest', 'ForeignMAValue']:
    ofdi_at_df.loc[:, f'{key}_at'] = ofdi_at_df[key] / ofdi_at_df.TotalAsset
    ofdi_at_df.loc[:, f'{key}_lat'] = ofdi_at_df[key] / ofdi_at_df.TotalAsset_1


In [71]:
ofdi_at_df.to_pickle(os.path.join(const.TEMP_PATH, 'firm_foreign_activities.pkl'))

# Merge OFDI data with regression data

In [35]:
ofdi_at_df: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, 'firm_foreign_activities.pkl')).drop(
    ['TotalAsset', 'TotalAsset_1'], axis=1)
reg_df: DataFrame = pd.read_stata(os.path.join(const.OUTPUT_PATH, '20241027_cc_reg_data.dta'))

In [37]:
tmp_ofdi_df: DataFrame = ofdi_at_df.copy()
tmp_ofdi_df[const.YEAR] -= 1
reg_df2: DataFrame = reg_df.merge(ofdi_at_df, on=[const.TICKER, const.YEAR], how='left').merge(
    tmp_ofdi_df, on=[const.TICKER, const.YEAR], how='left', suffixes=('', '_1'))

In [10]:
reg_df2.shape

(41072, 330)

In [41]:
tmp_ofdi_df.keys()

Index(['tic', 'year', 'ForeignEarnings', 'ForeignEarningsProportion',
       'ForeignCosts', 'ForeignMargin', 'ForeignMarginRatio',
       'ForeignMarginRate', 'InsInvestorProp', 'StatesharesProp',
       'ForeignInvestorProp', 'SubFirmAmount', 'BRCountryNum', 'TaxHavenNum',
       'SubFirmAT', 'SubFirmOE', 'SubFirmProfit', 'SubFirmInvest',
       'SubFirmNum', 'ForeignMAValue', 'ForeignMANum', 'ForeignEarnings_at',
       'ForeignEarnings_lat', 'ForeignCosts_at', 'ForeignCosts_lat',
       'ForeignMargin_at', 'ForeignMargin_lat', 'SubFirmAmount_at',
       'SubFirmAmount_lat', 'SubFirmAT_at', 'SubFirmAT_lat', 'SubFirmOE_at',
       'SubFirmOE_lat', 'SubFirmProfit_at', 'SubFirmProfit_lat',
       'SubFirmInvest_at', 'SubFirmInvest_lat', 'ForeignMAValue_at',
       'ForeignMAValue_lat'],
      dtype='object')

In [38]:
for key in tmp_ofdi_df.keys():
    if key in {const.TICKER, const.YEAR}:
        continue
    reg_df2.loc[reg_df2[key].notnull(), key] = winsorize(reg_df2[key].dropna(), limits=(0.01, 0.01))
    reg_df2.loc[reg_df2[f'{key}_1'].notnull(), f'{key}_1'] = winsorize(reg_df2[f'{key}_1'].dropna(), limits=(0.01, 0.01))
    reg_df2.loc[:, f'{key}_01'] = reg_df2[f'{key}_1'].fillna(reg_df2[key]).fillna(0)

In [39]:
reg_df2.dropna(subset=['size', 'TobinQ', 'OCF_lat', 'lev', 'top1', 'sale_growth', 'CAPEX_lat_1', 'soe'], how='any', inplace=True)
reg_df3: DataFrame = reg_df2[reg_df2[const.YEAR].apply(lambda x: 2009 < x < 2019)].copy()
reg_df3['Post_has_guarantee'] = reg_df3['Post2014'] * reg_df3['has_guarantee']
reg_df3['Post_NumGuarantee'] = reg_df3['Post2014'] * reg_df3['NumGuarantee']

In [40]:
reg_df3.to_stata(os.path.join(const.OUTPUT_PATH, '20241103_cc_reg_data.dta'), version=119, write_index=False)

## Append foreign shareholder information

In [42]:
with zipfile.ZipFile(os.path.join(const.CSMAR_PATH, '股东', '上市公司股本结构文件.zip'), 'r') as zip_ref:
    with zip_ref.open('HLD_Capstru.csv') as csv_file:
        hld_df: DataFrame = pd.read_csv(csv_file).dropna(how='any')
        hld_df['Reptdt'] = pd.to_datetime(hld_df['Reptdt'])
        hld_df.loc[:, const.YEAR] = hld_df['Reptdt'].dt.year
        hld_df['ForeignInvestorProp'] = hld_df['Nshrlpf'] / hld_df['Nshrttl']

In [45]:
hld_df.head()

,Stkcd,Reptdt,Nshrttl,Nshrlpf,year,ForeignInvestorProp
0,1,2003-03-31,1.945822e+09,0.0,2003,0.0
1,1,2003-06-30,1.945822e+09,0.0,2003,0.0
2,1,2003-09-30,1.945822e+09,0.0,2003,0.0
3,1,2003-12-31,1.945822e+09,0.0,2003,0.0
4,1,2004-03-31,1.945822e+09,0.0,2004,0.0


In [49]:
hld_valid = hld_df.loc[hld_df['ForeignInvestorProp'] > 0].rename(columns={'Stkcd': const.TICKER})[[
    const.TICKER, const.YEAR, 'ForeignInvestorProp']].drop_duplicates(subset=[const.TICKER, const.YEAR], keep='last')

In [46]:
reg_drop_keys = 'ForeignInvestorProp_01 StatesharesProp_01 ForeignInvestorProp_1 StatesharesProp_1 ForeignInvestorProp StatesharesProp'.split(' ')

In [50]:
reg_df3.drop(reg_drop_keys, axis=1, inplace=True)
tmp_hld = hld_valid.copy()
tmp_hld[const.YEAR] -= 1

reg_df4: DataFrame = reg_df3.merge(hld_valid, on=[const.TICKER, const.YEAR], how='left').merge(
    tmp_hld, on=[const.TICKER, const.YEAR], how='left', suffixes=('', '_1'))
reg_df4['ForeignInvestorProp'] = reg_df4['ForeignInvestorProp'].fillna(0)
reg_df4['ForeignInvestorProp_1'] = reg_df4['ForeignInvestorProp_1'].fillna(0)

In [56]:
reg_df4.to_stata(os.path.join(const.OUTPUT_PATH, '20241103_cc_reg_data.dta'), version=119, write_index=False)

In [54]:
log_Keys = 'SubFirmAmount_1 SubFirmAT_1 SubFirmOE_1 SubFirmProfit_1 SubFirmInvest_1 ForeignEarnings_1 ForeignCosts_1'.split(' ')
log_Keys

['SubFirmAmount_1',
 'SubFirmAT_1',
 'SubFirmOE_1',
 'SubFirmProfit_1',
 'SubFirmInvest_1',
 'ForeignEarnings_1',
 'ForeignCosts_1']

In [55]:
for key in log_Keys:
    reg_df4.loc[:, f'ln_{key}'] = reg_df4[key].apply(lambda x: np.log(x + 1))

C:\Users\wangy\AppData\Local\Temp\ipykernel_30216\2843661264.py:2: RuntimeWarning: invalid value encountered in log
  reg_df4.loc[:, f'ln_{key}'] = reg_df4[key].apply(lambda x: np.log(x + 1))
